In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("pollution_us_2000_2016.csv")
df.head()

df = df.drop(columns=['Unnamed: 0', 'Site Num', 'Address', 'City'])
pd.unique(df['O3 Units'])

df.isna().sum()

d = df.dropna()
df['SO2 AQI'] = df.groupby(['County', 'Date Local'])['SO2 AQI'].ffill()
df['CO AQI'] = df.groupby(['County', 'Date Local'])['CO AQI'].bfill()
df.isna().sum()
df = df.dropna()
df.isna().sum()
df.to_csv("pollution.csv")

d = pd.read_csv("pollution.csv")

d

,Unnamed: 0,State Code,County Code,State,County,Date Local,NO2 Units,NO2 Mean,NO2 1st Max Value,NO2 1st Max Hour,...,SO2 Units,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Units,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
0,0,4,13,Arizona,Maricopa,2000-01-01,Parts per billion,19.041667,49.0,19,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,1.145833,4.200,21,25.0
1,1,4,13,Arizona,Maricopa,2000-01-01,Parts per billion,19.041667,49.0,19,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,0.878947,2.200,23,25.0
2,2,4,13,Arizona,Maricopa,2000-01-01,Parts per billion,19.041667,49.0,19,...,Parts per billion,2.975000,6.6,23,13.0,Parts per million,1.145833,4.200,21,25.0
3,3,4,13,Arizona,Maricopa,2000-01-01,Parts per billion,19.041667,49.0,19,...,Parts per billion,2.975000,6.6,23,13.0,Parts per million,0.878947,2.200,23,25.0
4,4,4,13,Arizona,Maricopa,2000-01-02,Parts per billion,22.958333,36.0,19,...,Parts per billion,1.958333,3.0,22,4.0,Parts per million,0.850000,1.600,23,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1746339,1746656,56,21,Wyoming,Laramie,2016-03-30,Parts per billion,1.083333,1.6,9,...,Parts per billion,0.000000,0.0,2,0.0,Parts per million,0.091667,0.100,2,1.0
1746340,1746657,56,21,Wyoming,Laramie,2016-03-31,Parts per billion,0.939130,1.3,5,...,Parts per billion,-0.022727,0.0,0,0.0,Parts per million,0.067714,0.127,0,1.0
1746341,1746658,56,21,Wyoming,Laramie,2016-03-31,Parts per billion,0.939130,1.3,5,...,Parts per billion,-0.022727,0.0,0,0.0,Parts per million,0.100000,0.100,0,1.0
1746342,1746659,56,21,Wyoming,Laramie,2016-03-31,Parts per billion,0.939130,1.3,5,...,Parts per billion,0.000000,0.0,5,0.0,Parts per million,0.067714,0.127,0,1.0


In [4]:

df = pd.read_csv("pollution.csv")
UniqueID = []
FIPS = []

dfsub = df.loc[:,['State', 'County', 'State Code', 'County Code']]
dfloc = dfsub.astype({'State Code': str, 'County Code': str})

# find fips in pollution
dfloc = dfloc.replace({'State Code': {'0': '0', '1': '01', '2': '02', '3': '03', '4': '04', '5': '05', '6': '06', '7': '07', '8': '08', '9':'09'}})
fips_list = []
for i in dfloc['County Code']:
  if int(i) < 10:
    fips_list.append("00" + i)
  elif int(i) < 100:
    fips_list.append("0" + i)
  else:
    fips_list.append(i)
dfloc['County Code'] = fips_list

FIPS = dfloc['State Code'] + dfloc['County Code']
# Find date table in PD

dfdate = df.loc[:,['State', 'Date Local']]
dfdate = dfdate.loc[:,['Date Local']]
ymd = []


for info in dfdate['Date Local']:
  info_str = str(info)
  info_split = info_str.split('-')
  ymd.append(info_split[0] + info_split[1] + info_split[2])

for i in range(len(df)):
  UniqueID.append(i+1)

UniqueID = np.array(UniqueID)
FIPS = np.array(FIPS)
print(FIPS)
ymd = np.array(ymd)
dfdate['ymd'] = ymd



df['UniqueID'] = UniqueID
df['FIPS'] = FIPS
df['YMD'] = dfdate['ymd']
 
SO2 = df[['UniqueID','SO2 AQI','SO2 Mean', 'SO2 1st Max Value', 'SO2 1st Max Hour', 'FIPS','YMD']]

NO2 = df[['UniqueID','NO2 AQI','NO2 Mean', 'NO2 1st Max Value', 'NO2 1st Max Hour','FIPS','YMD']]

CO =  df[['UniqueID','CO AQI','CO Mean', 'CO 1st Max Value', 'CO 1st Max Hour','FIPS','YMD']]

O3 = df[['UniqueID','O3 AQI','O3 Mean', 'O3 1st Max Value', 'O3 1st Max Hour','FIPS','YMD']]

SO2.to_csv("SO2.csv")
NO2.to_csv("NO2.csv")
O3.to_csv("O3.csv")
CO.to_csv("CO.csv")

CO


['04013' '04013' '04013' ... '56021' '56021' '56021']


,UniqueID,CO AQI,CO Mean,CO 1st Max Value,CO 1st Max Hour,FIPS,YMD
0,1,25.0,1.145833,4.200,21,04013,20000101
1,2,25.0,0.878947,2.200,23,04013,20000101
2,3,25.0,1.145833,4.200,21,04013,20000101
3,4,25.0,0.878947,2.200,23,04013,20000101
4,5,26.0,0.850000,1.600,23,04013,20000102
...,...,...,...,...,...,...,...
1746339,1746340,1.0,0.091667,0.100,2,56021,20160330
1746340,1746341,1.0,0.067714,0.127,0,56021,20160331
1746341,1746342,1.0,0.100000,0.100,0,56021,20160331
1746342,1746343,1.0,0.067714,0.127,0,56021,20160331
